In [1]:
"""
Created on Fri Jan 27 15:18 2023

Look at the results of the test of the NN on Smith data

Author: @claraburgard

"""

'\nCreated on Fri Jan 27 15:18 2023\n\nLook at the results of the test of the NN on Smith data\n\nAuthor: @claraburgard\n\n'

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import glob

In [2]:
%matplotlib qt5

QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-burgardc'


DEFINE OPTIONS

In [3]:
TS_opt = 'extrap' # extrap, whole, thermocline
norm_method =  'std' # std, interquart, minmax
exp_name = 'newbasic2' #'allbutconstants' #'onlyTSdraftandslope' #'TSdraftbotandiceddandwcd' #'onlyTSisfdraft' #'TSdraftbotandiceddandwcdreldGL' #TSdraftslopereldGL

RESULTS CROSS-VALIDATION CLASSIC

In [4]:
home_path = '/bettik/burgardc/'

run_list = ['OPM006','OPM016','OPM018','OPM021']

diff_Gt_CVtime_list = []
diff_box1_CVtime_list = []
diff_Gt_CVisf_list = []
diff_box1_CVisf_list = []

ref_Gt_list = []
ref_box1_list = []
Gt_CVtime_list = []
Gt_CVisf_list = []


for n,nemo_run in enumerate(run_list):

    outputpath_melt = '/bettik/burgardc/DATA/BASAL_MELT_PARAM/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'

    ### READ IN THE REFERENCE
    NEMO_melt_rates_1D = xr.open_dataset(outputpath_melt+'melt_rates_1D_NEMO_oneFRIS.nc')
    ref_Gt = NEMO_melt_rates_1D['melt_Gt_per_y_tot']
    ref_Gt_list.append(ref_Gt)
    NEMO_box1_myr = xr.open_dataset(outputpath_melt+'melt_rates_box1_NEMO_oneFRIS.nc')
    ref_box1 = NEMO_box1_myr['mean_melt_box1_myr']
    ref_box1_list.append(ref_box1)
    
    ### READ IN THE PARAM FILES - CV TIME
    
    # Param files
    outputpath_melt = home_path+'DATA/BASAL_MELT_PARAM/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'
    melt_param_files_CVtime = list(sorted(glob.glob(outputpath_melt+'eval_metrics_1D_*_CVtime.nc')))
    
    param_list = []
    for mfilename in melt_param_files_CVtime:
        paramname = mfilename[91:157].split('.')[0].split('_')[:-2]
        #paramname = mfilename[91:157].split('.')[0]
        #paramname = mfilename[105:157].split('.')[0].split('_')[:-1]
        mparam = '_'.join(paramname)
        param_list.append(mparam)
    
    ds_melt_param_CVtime  = xr.open_mfdataset(melt_param_files_CVtime, concat_dim='new_param', combine='nested', coords='minimal',compat='override')#, chunks={'x': chunksize, 'y': chunksize})
    if 'option' in ds_melt_param_CVtime.coords:        
        ds_melt_param_CVtime = ds_melt_param_CVtime.drop('param').drop('option')
    else:
        ds_melt_param_CVtime = ds_melt_param_CVtime.drop('param')
    ds_melt_param_CVtime = ds_melt_param_CVtime.rename({'new_param': 'param'})
    ds_melt_param_CVtime = ds_melt_param_CVtime.assign_coords(param=param_list)
    Gt_CVtime_list.append(ds_melt_param_CVtime['melt_1D_Gt_per_y'])
    diff_Gt_CVtime = ds_melt_param_CVtime['melt_1D_Gt_per_y'] - ref_Gt
    diff_Gt_CVtime = diff_Gt_CVtime.assign_coords({'time': np.arange(1,len(diff_Gt_CVtime.time)+1)+n*50})
    diff_Gt_CVtime_list.append(diff_Gt_CVtime)
    diff_box1_CVtime = ds_melt_param_CVtime['melt_1D_mean_myr_box1'].mean('time') - ref_box1.mean('time')
    diff_box1_CVtime_list.append(diff_box1_CVtime)
    
    melt_param_files_CVisf = list(sorted(glob.glob(outputpath_melt+'eval_metrics_1D_*_CVshelves.nc')))
    ds_melt_param_CVisf  = xr.open_mfdataset(melt_param_files_CVisf, concat_dim='new_param', combine='nested', coords='minimal',compat='override')#, chunks={'x': chunksize, 'y': chunksize})
    if 'option' in ds_melt_param_CVisf.coords:        
        ds_melt_param_CVisf  = ds_melt_param_CVisf.drop('param').drop('option')
    else:
        ds_melt_param_CVisf  = ds_melt_param_CVisf.drop('param')
    ds_melt_param_CVisf = ds_melt_param_CVisf.rename({'new_param': 'param'})
    ds_melt_param_CVisf = ds_melt_param_CVisf.assign_coords(param=param_list)
    Gt_CVisf_list.append(ds_melt_param_CVisf['melt_1D_Gt_per_y'])
    diff_Gt_CVisf = ds_melt_param_CVisf['melt_1D_Gt_per_y'] - ref_Gt
    diff_Gt_CVisf = diff_Gt_CVisf.assign_coords({'time': np.arange(1,len(diff_Gt_CVisf.time)+1)+n*50})
    diff_Gt_CVisf_list.append(diff_Gt_CVisf)
    diff_box1_CVisf = ds_melt_param_CVisf['melt_1D_mean_myr_box1'].mean('time') - ref_box1.mean('time')
    diff_box1_CVisf_list.append(diff_box1_CVisf)
        

Gt_all_CVtime = xr.concat(Gt_CVtime_list, dim='nemo_run')
Gt_all_CVisf = xr.concat(Gt_CVisf_list, dim='nemo_run')
diff_Gt_all_CVtime = xr.concat(diff_Gt_CVtime_list, dim='time')
diff_box1_all_CVtime = xr.concat(diff_box1_CVtime_list, dim='nemo_run')   
diff_Gt_all_CVisf = xr.concat(diff_Gt_CVisf_list, dim='time')
diff_box1_all_CVisf = xr.concat(diff_box1_CVisf_list, dim='nemo_run')   

RESULTS CROSS-VALIDATION NN

In [5]:
run_list = ['OPM006','OPM016','OPM018','OPM021']

diff_Gt_CVtime_list = []
diff_box1_CVtime_list = []
diff_Gt_CVisf_list = []
diff_box1_CVisf_list = []

ref_Gt_list = []
ref_box1_list = []
Gt_CVtime_list = []
Gt_CVisf_list = []

outputpath_melt = '/bettik/burgardc/DATA/NN_PARAM/processed/MELT_RATE/'

for n,nemo_run in enumerate(run_list):

    ### CV TIME
    melt_param_files_CVtime = list(sorted(glob.glob(outputpath_melt+'CV_TBLOCKS/evalmetrics_1D_CV_*newbasic*_normstd_'+nemo_run+'.nc')))
    
    param_list = []
    for mfilename in melt_param_files_CVtime:
        paramname = mfilename[80:157].split('.')[0].split('_')[:-2]
        #paramname = mfilename[80:157].split('.')[0].split('_')[:-3]
        #paramname = mfilename[79:157].split('.')[0].split('_')[:-3]
        mparam = '_'.join(paramname)
        param_list.append(mparam)
        
    ds_melt_param_CVtime  = xr.open_mfdataset(melt_param_files_CVtime, concat_dim='param', combine='nested', coords='minimal',compat='override')#, chunks={'x': chunksize, 'y': chunksize})
    ds_melt_param_CVtime = ds_melt_param_CVtime.assign_coords(param=param_list)
    Gt_CVtime_list.append(ds_melt_param_CVtime['predicted_melt'].sel(metrics='Gt'))
    diff_Gt_CVtime = (ds_melt_param_CVtime['predicted_melt'] - ds_melt_param_CVtime['reference_melt']).sel(metrics='Gt')
    diff_Gt_CVtime = diff_Gt_CVtime.assign_coords({'time': np.arange(1,len(diff_Gt_CVtime.time)+1)+n*50})
    diff_Gt_CVtime_list.append(diff_Gt_CVtime)
    diff_box1_CVtime = (ds_melt_param_CVtime['predicted_melt'].mean('time') - ds_melt_param_CVtime['reference_melt'].mean('time')).sel(metrics='box1')
    diff_box1_CVtime_list.append(diff_box1_CVtime)
    ref_Gt_list.append(ds_melt_param_CVtime['reference_melt'].isel(param=0).sel(metrics='Gt'))
    ref_box1_list.append(ds_melt_param_CVtime['reference_melt'].isel(param=0).sel(metrics='box1'))

    ### CV ISF
    melt_param_files_CVisf = list(sorted(glob.glob(outputpath_melt+'CV_ISF/evalmetrics_1D_CV_*newbasic*_normstd_'+nemo_run+'.nc')))
        
    ds_melt_param_CVisf  = xr.open_mfdataset(melt_param_files_CVisf, concat_dim='param', combine='nested', coords='minimal',compat='override')#, chunks={'x': chunksize, 'y': chunksize})
    ds_melt_param_CVisf = ds_melt_param_CVisf.assign_coords(param=param_list)
    Gt_CVisf_list.append(ds_melt_param_CVisf['predicted_melt'].sel(metrics='Gt'))
    diff_Gt_CVisf = (ds_melt_param_CVisf['predicted_melt'] - ds_melt_param_CVisf['reference_melt']).sel(metrics='Gt')
    diff_Gt_CVisf = diff_Gt_CVisf.assign_coords({'time': np.arange(1,len(diff_Gt_CVisf.time)+1)+n*50})
    diff_Gt_CVisf_list.append(diff_Gt_CVisf)
    diff_box1_CVisf = (ds_melt_param_CVisf['predicted_melt'].mean('time') - ds_melt_param_CVisf['reference_melt'].mean('time')).sel(metrics='box1')
    diff_box1_CVisf_list.append(diff_box1_CVisf)

Gt_all_CVtime_NN = xr.concat(Gt_CVtime_list, dim='nemo_run')
Gt_all_CVisf_NN = xr.concat(Gt_CVisf_list, dim='nemo_run')
diff_Gt_all_CVtime_NN = xr.concat(diff_Gt_CVtime_list, dim='time')
diff_box1_all_CVtime_NN = xr.concat(diff_box1_CVtime_list, dim='nemo_run')   
diff_Gt_all_CVisf_NN = xr.concat(diff_Gt_CVisf_list, dim='time')
diff_box1_all_CVisf_NN = xr.concat(diff_box1_CVisf_list, dim='nemo_run')   

In [6]:
ref_melt = xr.concat(ref_Gt_list, dim='nemo_run')

In [7]:
for nrun in tqdm(Gt_all_CVtime_NN.nemo_run):
    
    plt.figure()
    
    if nrun == 0:
        ccolor = 'magenta'
    elif nrun == 1:
        ccolor = 'maroon'
    elif nrun == 2:
        ccolor = 'gold'
    elif nrun == 3:
        ccolor = 'red'
        
    for mparam in Gt_all_CVtime_NN.param:
        Gt_all_CVtime_NN.sel(param=mparam,nemo_run=nrun).sum('Nisf').plot(color=ccolor,alpha=0.2)
        Gt_all_CVisf_NN.sel(param=mparam,nemo_run=nrun).sum('Nisf').plot(color=ccolor,alpha=0.2)

    for mparam in Gt_all_CVtime.param:
        Gt_all_CVtime.sel(param=mparam,nemo_run=nrun,profile_domain=50).sum('Nisf').plot(color=ccolor,alpha=0.2)
        Gt_all_CVisf.sel(param=mparam,nemo_run=nrun,profile_domain=50).sum('Nisf').plot(color=ccolor,alpha=0.2)
    
    ref_melt.sel(nemo_run=nrun).sum('Nisf').plot(color=ccolor)

  0%|          | 0/4 [00:00<?, ?it/s]

libGL error: No matching fbConfigs or visuals found
libGL error: failed to load driver: swrast


In [8]:
diff_Gt_all_param_together_CVtime = xr.concat([diff_Gt_all_CVtime.sel(profile_domain=50),diff_Gt_all_CVtime_NN], dim='param')
diff_Gt_all_param_together_CVisf = xr.concat([diff_Gt_all_CVisf.sel(profile_domain=50),diff_Gt_all_CVisf_NN], dim='param')

In [9]:
diff_box1_all_param_together_CVtime = xr.concat([diff_box1_all_CVtime.sel(profile_domain=50),diff_box1_all_CVtime_NN], dim='param')
diff_box1_all_param_together_CVisf = xr.concat([diff_box1_all_CVisf.sel(profile_domain=50),diff_box1_all_CVisf_NN], dim='param')

In [10]:
special_nisf_list = [11, 69, 43, 12, 70, 44, 13, 58, 71, 45, 30, 31, 61, 73, 47, 48, 33, 18, 10, 65, 51, 22, 38, 52, 23, 66, 53, 39, 24,
                     40, 54, 75, 25, 42, 55]

In [12]:
RMSE_Gt_all_CVtime = np.sqrt((diff_Gt_all_param_together_CVtime**2).sel(Nisf=special_nisf_list).mean(['time','Nisf']))
RMSE_box1_all_CVtime = np.sqrt((diff_box1_all_param_together_CVtime**2).sel(Nisf=special_nisf_list).mean(['nemo_run','Nisf']))
RMSE_Gt_all_CVisf = np.sqrt((diff_Gt_all_param_together_CVisf**2).sel(Nisf=special_nisf_list).mean(['time','Nisf']))
RMSE_box1_all_CVisf = np.sqrt((diff_box1_all_param_together_CVisf**2).sel(Nisf=special_nisf_list).mean(['nemo_run','Nisf']))

In [13]:
#isf_dim = [10,11,12,13,18,22,23,25,30,31,33,39,40,42,43,44,45,47,51,55,58,61,65,66,69,70,71,73,75]

RESULTS NN MODELS

In [14]:
res_1D_nrun_list = []
for nemo_run in ['bf663', 'bi646']: #'bi646'
    inputpath_mask='/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/'
    outputpath_melt_nn = '/bettik/burgardc/DATA/NN_PARAM/processed/MELT_RATE/'
    outputpath_melt = '/bettik/burgardc/DATA/NN_PARAM/processed/MELT_RATE/SMITH_'+nemo_run+'_CLASSIC/'
    inputpath_data='/bettik/burgardc/DATA/NN_PARAM/interim/SMITH_'+nemo_run+'/'
    plot_path = '/bettik/burgardc/PLOTS/NN_plots/1D_eval/'

    res_1D_mods_list = []
    for mod_size in ['small']:
            outputpath_melt_nn = '/bettik/burgardc/DATA/NN_PARAM/processed/MELT_RATE/SMITH_'+nemo_run+'/'
            res_1D_all_list = []
            for yy in range(1980, 1980 + 60):
                res_1D_yy = xr.open_dataset(outputpath_melt_nn + 'evalmetrics_1D_'+mod_size+'_'+exp_name+'_ensmean_'+TS_opt+'_norm'+norm_method+'_'+str(yy)+'_'+nemo_run+'.nc')
                res_1D_all_list.append(res_1D_yy.assign_coords({'time': yy}))
            res_1D_all_xr = xr.concat(res_1D_all_list, dim='time')
            res_1D_mods_list.append(res_1D_all_xr.assign_coords({'param': mod_size}))
    res_1D_mods_all = xr.concat(res_1D_mods_list, dim='param').assign_coords(nemo_run=nemo_run)
    res_1D_nrun_list.append(res_1D_mods_all)
res_1D_nrun = xr.concat(res_1D_nrun_list, dim='nemo_run')

In [15]:
nn_melt_Gt_all = res_1D_nrun['predicted_melt'].sel(metrics='Gt')
nn_melt_box1_all = res_1D_nrun['predicted_melt'].sel(metrics='box1')

RESULTS CLASSIC PARAM

In [16]:

res_1D_nrun_list = []
run_list = ['bf663','bi646'] 

target_Gt_list = []
ds_Gt_list = []
ds_box1_list = []
target_box1_list = []

for nemo_run in run_list:
    file_isf_list = []

    target_melt_list = []
    ds_melt_Gt_list = []
    ds_melt_box1_list = []
    ref_box1_list = []
    
    inputpath_mask='/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/'
    outputpath_melt_nn = '/bettik/burgardc/DATA/NN_PARAM/processed/MELT_RATE/'
    outputpath_melt = '/bettik/burgardc/DATA/NN_PARAM/processed/MELT_RATE/SMITH_'+nemo_run+'_CLASSIC/'
    inputpath_data='/bettik/burgardc/DATA/NN_PARAM/interim/SMITH_'+nemo_run+'/'
    plot_path = '/bettik/burgardc/PLOTS/NN_plots/1D_eval/'

    for yy in tqdm(range(1980,1980 + 60)):

        # Param files
        melt_param_files = list(sorted(glob.glob(outputpath_melt+'eval_metrics_1D_*_oneFRIS_'+str(yy)+'.nc')))

        param_list = []
        for mfilename in melt_param_files:
            #paramname = mfilename[97:157].split('.')[0].split('_')[:-1]
            paramname = mfilename[87:157].split('.')[0].split('_')[:-2]
            #paramname = mfilename[105:157].split('.')[0].split('_')[:-1]
            mparam = '_'.join(paramname)
            param_list.append(mparam)

        ds_melt_param  = xr.open_mfdataset(melt_param_files, concat_dim='new_param', combine='nested', coords='minimal',compat='override')#, chunks={'x': chunksize, 'y': chunksize})
        if 'option' in ds_melt_param.coords:        
            ds_melt_param = ds_melt_param.drop('option')
        if 'calib' in ds_melt_param.coords: 
            ds_melt_param = ds_melt_param.drop('calib')
        if 'param' in ds_melt_param.coords: 
            ds_melt_param = ds_melt_param.drop('param')

        ds_melt_param = ds_melt_param.rename({'new_param': 'param'})
        ds_melt_param = ds_melt_param.assign_coords(param=param_list, time=yy)
        ds_melt_param_Gt = ds_melt_param['melt_1D_Gt_per_y'].sel(Nisf=ds_melt_param.Nisf)
        ds_melt_Gt_list.append(ds_melt_param_Gt)
        ds_melt_param_box1 = ds_melt_param['melt_1D_mean_myr_box1'].sel(Nisf=ds_melt_param.Nisf)
        ds_melt_box1_list.append(ds_melt_param_box1)

        # File for target
        NEMO_melt_rates_1D = xr.open_dataset(inputpath_data+'melt_rates_1D_NEMO_oneFRIS_'+str(yy)+'.nc')
        target_melt_Gt_yr = NEMO_melt_rates_1D['melt_Gt_per_y_tot'].sel(Nisf=ds_melt_param.Nisf)
        target_melt_list.append(target_melt_Gt_yr)
        NEMO_box1_myr = xr.open_dataset(inputpath_data+'melt_rates_box1_NEMO_oneFRIS_'+str(yy)+'.nc')
        ref_box1 = NEMO_box1_myr['mean_melt_box1_myr']
        ref_box1_list.append(ref_box1)
    
    target_melt_mod = xr.concat(target_melt_list, dim='time').assign_coords({'nemo_run': nemo_run})
    target_Gt_list.append(target_melt_mod)

    ref_box1_mod = xr.concat(ref_box1_list, dim='time').assign_coords({'nemo_run': nemo_run})
    target_box1_list.append(ref_box1_mod)

    ds_melt_Gt_mod = xr.concat(ds_melt_Gt_list, dim='time').assign_coords({'nemo_run': nemo_run})
    ds_Gt_list.append(ds_melt_Gt_mod)

    ds_melt_box1_mod = xr.concat(ds_melt_box1_list, dim='time').assign_coords({'nemo_run': nemo_run})
    ds_box1_list.append(ds_melt_box1_mod)

target_melt_all = xr.concat(target_Gt_list, dim='nemo_run')
target_box1_all = xr.concat(target_box1_list, dim='nemo_run')
ds_melt_Gt_all = xr.concat(ds_Gt_list, dim='nemo_run')
ds_melt_box1_all = xr.concat(ds_box1_list, dim='nemo_run')

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

In [ ]:
ds_melt_Gt_all.param

In [ ]:
ref_box1_all.sel(time=range(1980+30,1980+60)).mean()

In [ ]:
ref_box1_all.sel(time=range(1980,1980+30)).mean()

In [ ]:
target_melt_all.sel(time=range(1980,1980+60)).mean()

In [ ]:
ds_melt_Gt_all.sel(time=range(1980,1980+30)).sum('Nisf').mean('time').load()

In [17]:
nisf_of_int = res_1D_nrun.Nisf.drop_sel(Nisf=4)

In [19]:
param_melt_Gt_all = xr.concat([ds_melt_Gt_all,nn_melt_Gt_all], dim='param').load()
param_melt_box1_all = xr.concat([ds_melt_box1_all,nn_melt_box1_all], dim='param').load()

In [31]:
plt.figure()

for mparam in param_melt_Gt_all.param:
    param_melt_Gt_all.sel(param=mparam).sum('Nisf').plot(color='grey',alpha=0.2)

target_melt_all.sum('Nisf').plot(color='k')

In [ ]:
for mparam in param_list_of_int:
    print(mparam, param_melt_Gt_all.sel(time=range(1980,1980+30)).mean(['time','Nisf']).sel(param=mparam).values)
    print(mparam, param_melt_Gt_all.sel(time=range(1980+30,1980+60)).mean(['time','Nisf']).sel(param=mparam).values)

In [ ]:
for mparam in param_list_of_int:
    print(mparam, param_melt_Gt_all.sel(time=range(1980,1980+30)).sum('Nisf').mean('time').sel(param=mparam).values)
    print(mparam, param_melt_Gt_all.sel(time=range(1980+30,1980+60)).sum('Nisf').mean('time').sel(param=mparam).values)

In [20]:
diff_Gt_all_param = param_melt_Gt_all - target_melt_all

In [21]:
RMSE_Gt_all_first30 = np.sqrt((diff_Gt_all_param.sel(time=range(1980,1980+30))**2).sel(Nisf=nisf_of_int).mean(['time','Nisf']))
RMSE_Gt_all_last30 = np.sqrt((diff_Gt_all_param.sel(time=range(1980+30,1980+60))**2).sel(Nisf=nisf_of_int).mean(['time','Nisf']))
RMSE_Gt_all_60yy = np.sqrt((diff_Gt_all_param.sel(time=range(1980,1980+60))**2).sel(Nisf=nisf_of_int).mean(['time','Nisf']))

In [ ]:
for mparam in param_list_of_int:
    print(mparam, RMSE_Gt_all_first30.sel(param=mparam).values)
    print(mparam, RMSE_Gt_all_last30.sel(param=mparam).values)

In [59]:
def NRMSE(RMSE, var):
    return RMSE/(var.max() - var.min())

In [ ]:
for mparam in param_list_of_int:
    print(mparam, NRMSE(RMSE_Gt_all_60yy.sel(param=mparam),target_melt_all.sel(time=range(1980,1980+60)).mean()).values)

In [23]:
diff_box1_all_param_first30 = param_melt_box1_all.sel(time=range(1980,1980+30)).mean('time') - target_box1_all.sel(time=range(1980,1980+30)).mean('time')
diff_box1_all_param_last30 = param_melt_box1_all.sel(time=range(1980+30,1980+60)).mean('time') - target_box1_all.sel(time=range(1980+30,1980+60)).mean('time')
diff_box1_all_param_60yy = param_melt_box1_all.sel(time=range(1980,1980+60)).mean('time') - target_box1_all.sel(time=range(1980,1980+60)).mean('time')

RMSE_box1_all_first30 = np.sqrt((diff_box1_all_param_first30**2).sel(Nisf=nisf_of_int).mean('Nisf'))
RMSE_box1_all_last30 = np.sqrt((diff_box1_all_param_last30**2).sel(Nisf=nisf_of_int).mean('Nisf'))
RMSE_box1_all_60yy = np.sqrt((diff_box1_all_param_60yy**2).sel(Nisf=nisf_of_int).mean('Nisf'))

In [ ]:
for mparam in param_list_of_int:
    #print(mparam, RMSE_box1_all_first30.sel(param=mparam).values)
    print(mparam, RMSE_box1_all_last30.sel(param=mparam).values)

In [ ]:
for mparam in param_list_of_int:
    print(mparam, NRMSE(RMSE_box1_all_60yy.sel(param=mparam),ref_box1_all.sel(time=range(1980,1980+60)).mean()).values)

In [24]:
param_list_of_int = [
                    #'linear_local',
                    'small',
                    'quadratic_local',
                    'quadratic_local_locslope',
                     'lazero19_2', 
                     'boxes_4_pismyes_picopno', 'boxes_4_pismno_picopyes', 

                    ]

param_list_of_int_maps = [
                    'xsmall96',
                    'small',
                    'large',
                    'quadratic_local',
                    'quadratic_local_locslope',
                     'lazero19_2', 
                     'boxes_4_pismyes_picopno', 
                    ]

In [ ]:
fig, axs = plt.subplots(1, 2,figsize=(8.24*1.25/2,8.24/2.5),sharey=True)

if nemo_run == 'bf663':
    ccolors = ['deepskyblue','blue']
elif nemo_run == 'bi646':    
    ccolors = ['hotpink','red']
    
#ccolors = ['steelblue','darkcyan','c','deepskyblue','skyblue','aquamarine','green','lightsteelblue','cyan','orange','magenta']

k = 0
#plt.figure()
for mparam in param_list_of_int[::-1]:

    if nemo_run == 'bf663':
        axs[0].scatter(RMSE_Gt_all_60yy.sel(param=mparam),mparam,marker='o',c='orange', s=50)
        axs[1].scatter(RMSE_box1_all_60yy.sel(param=mparam),mparam,marker='o',c='orange', s=50)
    else:
        axs[0].scatter(RMSE_Gt_all_first30.sel(param=mparam),mparam,marker='o',c=ccolors[0], s=50)
        axs[1].scatter(RMSE_box1_all_first30.sel(param=mparam),mparam,marker='o',c=ccolors[0], s=50)

        axs[0].scatter(RMSE_Gt_all_last30.sel(param=mparam),mparam,marker='o',c=ccolors[1], s=50)
        axs[1].scatter(RMSE_box1_all_last30.sel(param=mparam),mparam,marker='o',c=ccolors[1], s=50)

if nemo_run == 'bf663':
    axs[0].set_xlim(0,40)
    #axs[1].set_xlim(0,2.05)
    axs[1].set_xlim(0,0.6)
elif nemo_run == 'bi646':
    axs[0].set_xlim(0,680)
    #axs[1].set_xlim(0,2.05)
    axs[1].set_xlim(0,2)

sns.despine()
plt.savefig(plot_path+'RMSE_scatter_box1_Smith_classic_NN_newbasic2_'+nemo_run+'.pdf')

In [33]:
RMSE_Gt_all_CVisf.param

<xarray.DataArray 'param' (param: 35)>
array(['HUB_linear_local', 'HUB_quadratic_local_cavslope',
       'HUB_quadratic_local_locslope', 'HUB_quadratic_local',
       'HUB_quadratic_mixed_cavslope', 'HUB_quadratic_mixed_locslope',
       'HUB_quadratic_mixed_mean', 'boxes_1_pismyes_picopno',
       'boxes_2_pismyes_picopno', 'boxes_3_pismyes_picopno',
       'boxes_3_pismyes_picopyes', 'boxes_4_pismno_picopyes',
       'boxes_4_pismyes_picopno', 'lazero19_2', 'lazero19_modif2',
       'linear_local', 'quadratic_local_cavslope', 'quadratic_local_locslope',
       'quadratic_local', 'quadratic_mixed_cavslope',
       'quadratic_mixed_locslope', 'quadratic_mixed_mean',
       'extra_large_newbasic2_extrap', 'extra_large_newbasic_extrap',
       'large_newbasic2_extrap', 'large_newbasic_extrap',
       'medium_newbasic2_extrap', 'medium_newbasic_extrap',
       'mini_newbasic2_extrap', 'mini_newbasic_extrap',
       'small64_newbasic2_extrap', 'small_newbasic2_extrap',
       'small_newbasic_extrap', 'xsmall64_newbasic2_extrap',
       'xsmall96_newbasic2_extrap'], dtype='<U28')
Coordinates:
    metrics         <U2 'Gt'
    pism            <U3 'yes'
    profile_domain  int64 50
    box_nb_tot      int64 5
    config          int64 2
  * param           (param) <U28 'HUB_linear_local' ... 'xsmall96_newbasic2_e...

In [25]:
fig, axs = plt.subplots(1, 2,figsize=(8.24*1.25/2,8.24/2.5),sharey=True)

if nemo_run == 'bf663':
    ccolors = ['deepskyblue','blue']
elif nemo_run == 'bi646':    
    ccolors = ['hotpink','red']
    
#ccolors = ['steelblue','darkcyan','c','deepskyblue','skyblue','aquamarine','green','lightsteelblue','cyan','orange','magenta']

k = 0
#plt.figure()
for mparam in param_list_of_int[::-1]:
    

    axs[0].scatter(RMSE_Gt_all_60yy.sel(param=mparam,nemo_run='bf663'),mparam,marker='o',c='gold', s=50)
    axs[1].scatter(RMSE_box1_all_60yy.sel(param=mparam,nemo_run='bf663'),mparam,marker='o',c='gold', s=50)
        

    axs[0].scatter(RMSE_Gt_all_last30.sel(param=mparam,nemo_run='bi646'),mparam,marker='o',c=ccolors[1], s=50)
    axs[1].scatter(RMSE_box1_all_last30.sel(param=mparam,nemo_run='bi646'),mparam,marker='o',c=ccolors[1], s=50)
    

if nemo_run == 'bf663':
    axs[0].set_xlim(0,40)
    #axs[1].set_xlim(0,2.05)
    axs[1].set_xlim(0,0.6)
elif nemo_run == 'bi646':
    axs[0].set_xlim(0,680)
    #axs[1].set_xlim(0,2.05)
    axs[1].set_xlim(0,2.1)

#RMSE_Gt_all_CVisf    
#RMSE_box1_all_CVisf    
sns.despine()
#plt.savefig(plot_path+'RMSE_scatter_box1_Smith_classic_NN_newbasic2_'+nemo_run+'_forNEMOpresi.pdf')

In [64]:
target_box1_all.mean(['time','Nisf'])

<xarray.DataArray 'mean_melt_box1_myr' (nemo_run: 2)>
array([0.34259913, 0.8871872 ])
Coordinates:
    box_nb_tot  int64 5
    config      int64 2
  * nemo_run    (nemo_run) <U5 'bf663' 'bi646'

In [65]:
fig, axs = plt.subplots(1, 2,figsize=(8.24*1.25/2,8.24/2.5),sharey=True)

if nemo_run == 'bf663':
    ccolors = ['deepskyblue','blue']
elif nemo_run == 'bi646':    
    ccolors = ['hotpink','red']
    
#ccolors = ['steelblue','darkcyan','c','deepskyblue','skyblue','aquamarine','green','lightsteelblue','cyan','orange','magenta']


k = 0
#plt.figure()
for mparam in param_list_of_int[::-1]:
    

    #axs[0].scatter(NRMSE(RMSE_Gt_all_60yy.sel(param=mparam,nemo_run='bf663'),target_melt_all.sel(nemo_run='bf663')),mparam,marker='o',c='lightgrey', s=50)
    #axs[1].scatter(NRMSE(RMSE_box1_all_60yy.sel(param=mparam,nemo_run='bf663'),target_box1_all.sel(nemo_run='bf663')),mparam,marker='o',c='lightgrey', s=50)
    if mparam == 'small':
        axs[0].scatter(NRMSE(RMSE_Gt_all_60yy.sel(param=mparam,nemo_run='bi646'),target_melt_all.sel(nemo_run='bi646')),mparam,marker='o',c=ccolors[1], s=50)
        axs[1].scatter(NRMSE(RMSE_box1_all_60yy.sel(param=mparam,nemo_run='bi646'),target_box1_all.sel(nemo_run='bi646')),mparam,marker='o',c=ccolors[1], s=50)
    
    else:
        axs[0].scatter(NRMSE(RMSE_Gt_all_60yy.sel(param=mparam,nemo_run='bi646'),target_melt_all.sel(nemo_run='bi646')),mparam,marker='o',c=ccolors[1], s=50)
        axs[1].scatter(NRMSE(RMSE_box1_all_60yy.sel(param=mparam,nemo_run='bi646'),target_box1_all.sel(nemo_run='bi646')),mparam,marker='o',c=ccolors[1], s=50)
        
    axs[0].scatter(NRMSE(RMSE_Gt_all_60yy.sel(param=mparam,nemo_run='bi646'),target_melt_all.sel(nemo_run='bi646')),mparam,marker='o',c=ccolors[1], s=50)
    axs[1].scatter(NRMSE(RMSE_box1_all_60yy.sel(param=mparam,nemo_run='bi646'),target_box1_all.sel(nemo_run='bi646')),mparam,marker='o',c=ccolors[1], s=50)
    

if nemo_run == 'bf663':
    axs[0].set_xlim(0,40)
    #axs[1].set_xlim(0,2.05)
    axs[1].set_xlim(0,0.6)
elif nemo_run == 'bi646':
    axs[0].set_xlim(0,680)
    #axs[1].set_xlim(0,2.05)
    axs[1].set_xlim(0,2.1)

#RMSE_Gt_all_CVisf    
#RMSE_box1_all_CVisf    
sns.despine()
plt.savefig(plot_path+'RMSE_scatter_box1_Smith_classic_NN_newbasic2_'+nemo_run+'_forNEMOpresi.pdf')

In [ ]:
RMSE_box1_all_last30.sel(param='small')

In [ ]:
#NRMSE
fig, axs = plt.subplots(1, 2,figsize=(8.24*1.25/1.5,8.24/2),sharey=True)

if nemo_run == 'bf663':
    ccolors = ['deepskyblue','blue']
elif nemo_run == 'bi646':    
    ccolors = ['hotpink','red']
    
#ccolors = ['steelblue','darkcyan','c','deepskyblue','skyblue','aquamarine','green','lightsteelblue','cyan','orange','magenta']

k = 0
#plt.figure()
for mparam in param_list_of_int[::-1]:

    if nemo_run == 'bf663':
        axs[0].scatter(NRMSE(RMSE_Gt_all_60yy.sel(param=mparam), target_melt_all),mparam,marker='o',c='orange', s=50)
        axs[1].scatter(NRMSE(RMSE_box1_all_60yy.sel(param=mparam), ref_box1_all),mparam,marker='o',c='orange', s=50)
    else:
        axs[0].scatter(NRMSE(RMSE_Gt_all_first30.sel(param=mparam), target_melt_all),mparam,marker='o',c=ccolors[0], s=50)
        axs[1].scatter(NRMSE(RMSE_box1_all_first30.sel(param=mparam), ref_box1_all),mparam,marker='o',c=ccolors[0], s=50)

        axs[0].scatter(NRMSE(RMSE_Gt_all_last30.sel(param=mparam), target_melt_all),mparam,marker='o',c=ccolors[1], s=50)
        axs[1].scatter(NRMSE(RMSE_box1_all_last30.sel(param=mparam), ref_box1_all),mparam,marker='o',c=ccolors[1], s=50)

if nemo_run == 'bf663':
    axs[0].set_xlim(0,1)
    #axs[1].set_xlim(0,2.05)
    axs[1].set_xlim(0,1)
elif nemo_run == 'bi646':
    axs[0].set_xlim(0,0.4)
    #axs[1].set_xlim(0,2.05)
    axs[1].set_xlim(0,0.4)

sns.despine()
#plt.savefig(plot_path+'NRMSE_scatter_box1_Smith_classic_NN_newbasic2_'+nemo_run+'.pdf')

In [ ]:
fig, axs = plt.subplots(1, 2,figsize=(8.24*1.25/1.5,8.24/2),sharey=True)

if nemo_run == 'bf663':
    ccolors = ['deepskyblue','blue']
elif nemo_run == 'bi646':    
    ccolors = ['hotpink','red']
    
#ccolors = ['steelblue','darkcyan','c','deepskyblue','skyblue','aquamarine','green','lightsteelblue','cyan','orange','magenta']

k = 0
#plt.figure()
for mparam in param_list_of_int[::-1]:

    if nemo_run == 'bf663' or nemo_run == 'bi646' :
        axs[0].scatter(RMSE_Gt_all_60yy.sel(param=mparam),mparam,marker='o',c='red', s=50)
        axs[1].scatter(RMSE_box1_all_60yy.sel(param=mparam),mparam,marker='o',c='red', s=50)
    else:
        axs[0].scatter(RMSE_Gt_all_first30.sel(param=mparam),mparam,marker='o',c=ccolors[0], s=50)
        axs[1].scatter(RMSE_box1_all_first30.sel(param=mparam),mparam,marker='o',c=ccolors[0], s=50)

        axs[0].scatter(RMSE_Gt_all_last30.sel(param=mparam),mparam,marker='o',c=ccolors[1], s=50)
        axs[1].scatter(RMSE_box1_all_last30.sel(param=mparam),mparam,marker='o',c=ccolors[1], s=50)

if nemo_run == 'bf663':
    axs[0].set_xlim(0,40)
    #axs[1].set_xlim(0,2.05)
    axs[1].set_xlim(0,0.6)
elif nemo_run == 'bi646':
    axs[0].set_xlim(0,680)
    #axs[1].set_xlim(0,2.05)
    axs[1].set_xlim(0,2)

sns.despine()
plt.savefig(plot_path+'RMSE_scatter_box1_Smith_classic_NN_newbasic2_'+nemo_run+'_forOPENworkshop.pdf')

In [ ]:
RMSE_Gt_all_first30_wolargeones = np.sqrt((diff_Gt_all_param.sel(time=range(1980,1980+30))**2).sel(Nisf=nisf_of_int).drop_sel(Nisf=[10,11]).mean(['time','Nisf']))
RMSE_Gt_all_last30_wolargeones = np.sqrt((diff_Gt_all_param.sel(time=range(1980+30,1980+60))**2).sel(Nisf=nisf_of_int).drop_sel(Nisf=[10,11]).mean(['time','Nisf']))

RMSE_box1_all_first30_wolargeones = np.sqrt((diff_box1_all_param_first30**2).sel(Nisf=nisf_of_int).drop_sel(Nisf=[10,11]).mean('Nisf'))
RMSE_box1_all_last30_wolargeones = np.sqrt((diff_box1_all_param_last30**2).sel(Nisf=nisf_of_int).drop_sel(Nisf=[10,11]).mean('Nisf'))

In [ ]:
fig, axs = plt.subplots(1, 2,figsize=(8.24*1.25/1.5,8.24/2),sharey=True)

if nemo_run == 'bf663':
    ccolors = ['deepskyblue','blue']
elif nemo_run == 'bi646':    
    ccolors = ['hotpink','red']
    
#ccolors = ['steelblue','darkcyan','c','deepskyblue','skyblue','aquamarine','green','lightsteelblue','cyan','orange','magenta']

k = 0
#plt.figure()
for mparam in param_list_of_int[::-1]:

        
    axs[0].scatter(RMSE_Gt_all_first30_wolargeones.sel(param=mparam),mparam,marker='o',c=ccolors[0], s=50)
    axs[1].scatter(RMSE_box1_all_first30_wolargeones.sel(param=mparam),mparam,marker='o',c=ccolors[0], s=50)

    axs[0].scatter(RMSE_Gt_all_last30_wolargeones.sel(param=mparam),mparam,marker='o',c=ccolors[1], s=50)
    axs[1].scatter(RMSE_box1_all_last30_wolargeones.sel(param=mparam),mparam,marker='o',c=ccolors[1], s=50)

if nemo_run == 'bf663':
    axs[0].set_xlim(0,40)
    #axs[1].set_xlim(0,2.05)
    axs[1].set_xlim(0,0.6)
elif nemo_run == 'bi646':
    axs[0].set_xlim(0,110)
    #axs[1].set_xlim(0,2.05)
    axs[1].set_xlim(0,2)

sns.despine()
#plt.savefig(plot_path+'RMSE_scatter_box1_Smith_classic_NN_newbasic2_wolargeones_'+nemo_run+'.pdf')

In [ ]:
fig, axs = plt.subplots(1, 2,figsize=(8.24*1.25/1.5,8.24/2),sharey=True)

if nemo_run == 'bf663':
    ccolors = ['deepskyblue','blue']
elif nemo_run == 'bi646':    
    ccolors = ['hotpink','red']
    
#ccolors = ['steelblue','darkcyan','c','deepskyblue','skyblue','aquamarine','green','lightsteelblue','cyan','orange','magenta']

k = 0
#plt.figure()
for mparam in param_list_of_int[::-1]:

    if nemo_run == 'bf663':

        if mparam in ['xsmall96','small','large']:
            axs[0].scatter(RMSE_Gt_all_CVtime.sel(param=mparam+'_newbasic2_extrap'),mparam,marker='x',c='grey', s=50)
            axs[1].scatter(RMSE_box1_all_CVtime.sel(param=mparam+'_newbasic2_extrap'),mparam,marker='x',c='grey', s=50)

            axs[0].scatter(RMSE_Gt_all_CVisf.sel(param=mparam+'_newbasic2_extrap'),mparam,marker='+',c='grey', s=100)
            axs[1].scatter(RMSE_box1_all_CVisf.sel(param=mparam+'_newbasic2_extrap'),mparam,marker='+',c='grey', s=100)
        else:
            axs[0].scatter(RMSE_Gt_all_CVtime.sel(param=mparam),mparam,marker='x',c='grey', s=50)
            axs[1].scatter(RMSE_box1_all_CVtime.sel(param=mparam),mparam,marker='x',c='grey', s=50)

            axs[0].scatter(RMSE_Gt_all_CVisf.sel(param=mparam),mparam,marker='+',c='grey', s=100)
            axs[1].scatter(RMSE_box1_all_CVisf.sel(param=mparam),mparam,marker='+',c='grey', s=100)
            
        axs[0].scatter(RMSE_Gt_all_60yy.sel(param=mparam),mparam,marker='o',c='orange', s=50)
        axs[1].scatter(RMSE_box1_all_60yy.sel(param=mparam),mparam,marker='o',c='orange', s=50)
        
        
    else:
        axs[0].scatter(RMSE_Gt_all_first30.sel(param=mparam),mparam,marker='o',c=ccolors[0], s=50)
        axs[1].scatter(RMSE_box1_all_first30.sel(param=mparam),mparam,marker='o',c=ccolors[0], s=50)

        axs[0].scatter(RMSE_Gt_all_last30.sel(param=mparam),mparam,marker='o',c=ccolors[1], s=50)
        axs[1].scatter(RMSE_box1_all_last30.sel(param=mparam),mparam,marker='o',c=ccolors[1], s=50)

if nemo_run == 'bf663':
    axs[0].set_xlim(0,70)
    axs[1].set_xlim(0,2.05)
    #axs[1].set_xlim(0,2)
elif nemo_run == 'bi646':
    axs[0].set_xlim(0,680)
    #axs[1].set_xlim(0,2.05)
    axs[1].set_xlim(0,2)

sns.despine()
plt.savefig(plot_path+'RMSE_scatter_box1_Smith_classic_NN_withCV_newbasic2_'+nemo_run+'.pdf')

In [ ]:
if nemo_run == 'bf663':
    fig, axs = plt.subplots(1, 2,figsize=(8.24/2,8.24/2),sharey=True)
else:
    fig, axs = plt.subplots(1, 2,figsize=(8.24*1.25/1.5,8.24/2),sharey=True)
    
if nemo_run == 'bf663':
    ccolors = ['deepskyblue','blue']
elif nemo_run == 'bi646':    
    ccolors = ['hotpink','red']
    
#ccolors = ['steelblue','darkcyan','c','deepskyblue','skyblue','aquamarine','green','lightsteelblue','cyan','orange','magenta']

k = 0
#plt.figure()
for mparam in param_list_of_int_maps[::-1]:
    
    if mparam in ['xsmall96','small','large']:
        ccolor = 'orange'
    else:
        ccolor = 'lightskyblue'
        
    axs[0].scatter(RMSE_Gt_all_60yy.sel(param=mparam),mparam,marker='o',c=ccolor, s=50) ##0069B5
    axs[1].scatter(RMSE_box1_all_60yy.sel(param=mparam),mparam,marker='o',c=ccolor, s=50)


if nemo_run == 'bf663':
    axs[0].set_xlim(0,40)
    #axs[1].set_xlim(0,2.05)
    axs[1].set_xlim(0,0.6)
elif nemo_run == 'bi646':
    axs[0].set_xlim(0,680)
    #axs[1].set_xlim(0,2.05)
    axs[1].set_xlim(0,2)

sns.despine()
plt.savefig(plot_path+'RMSE_scatter_box1_Smith_classic_NN_newbasic2_'+nemo_run+'_woPICOP.pdf', transparent=True)

In [ ]:
fig, axs = plt.subplots(2, 1,figsize=(8.25*1.7, 8.25/2.25),sharex=True)

if nemo_run == 'bf663':
    ccolors = ['deepskyblue','blue']
elif nemo_run == 'bi646':    
    ccolors = ['hotpink','red']
    
#ccolors = ['steelblue','darkcyan','c','deepskyblue','skyblue','aquamarine','green','lightsteelblue','cyan','orange','magenta']

k = 0
#plt.figure()
for mparam in param_list_of_int_maps:

    if nemo_run == 'bf663':
        axs[0].scatter(mparam,RMSE_Gt_all_60yy.sel(param=mparam),marker='o',c='orange', s=50)
        axs[1].scatter(mparam,RMSE_box1_all_60yy.sel(param=mparam),marker='o',c='orange', s=50)
        axs[0].axhline(y=24,c='grey',linestyle='dotted',linewidth=1)
        axs[1].axhline(y=0.34,c='grey',linestyle='dotted',linewidth=1)
    else:
        axs[0].scatter(RMSE_Gt_all_first30.sel(param=mparam),mparam,marker='o',c=ccolors[0], s=50)
        axs[1].scatter(RMSE_box1_all_first30.sel(param=mparam),mparam,marker='o',c=ccolors[0], s=50)

        axs[0].scatter(RMSE_Gt_all_last30.sel(param=mparam),mparam,marker='o',c=ccolors[1], s=50)
        axs[1].scatter(RMSE_box1_all_last30.sel(param=mparam),mparam,marker='o',c=ccolors[1], s=50)

if nemo_run == 'bf663':
    axs[0].set_ylim(0,40)
    #axs[1].set_xlim(0,2.05)
    axs[1].set_ylim(0,0.6)
elif nemo_run == 'bi646':
    axs[0].set_xlim(0,680)
    #axs[1].set_xlim(0,2.05)
    axs[1].set_xlim(0,2)

sns.despine()
plt.savefig(plot_path+'RMSE_scatter_box1_Smith_classic_NN_newbasic2_'+nemo_run+'_horizontal.pdf')

In [ ]:
fig, axs = plt.subplots(1, 2,figsize=(8.24*1.25/1.5,8.24/2),sharey=True)

if nemo_run == 'bf663':
    ccolors = ['deepskyblue','blue']
elif nemo_run == 'bi646':    
    ccolors = ['red']
    
#ccolors = ['steelblue','darkcyan','c','deepskyblue','skyblue','aquamarine','green','lightsteelblue','cyan','orange','magenta']

k = 0
#plt.figure()
for mparam in param_list_of_int_maps[::-1]:

    axs[0].scatter(RMSE_Gt_all_60yy.sel(param=mparam),mparam,marker='o',c='red', s=50)
    axs[1].scatter(RMSE_box1_all_60yy.sel(param=mparam),mparam,marker='o',c='red', s=50)

if nemo_run == 'bf663':
    axs[0].set_xlim(0,40)
    #axs[1].set_xlim(0,2.05)
    axs[1].set_xlim(0,0.6)
elif nemo_run == 'bi646':
    axs[0].set_xlim(0,680)
    #axs[1].set_xlim(0,2.05)
    axs[1].set_xlim(0,2)

sns.despine()
plt.savefig(plot_path+'RMSE_scatter_box1_Smith_classic_NN_newbasic2_'+nemo_run+'_woPICOP.pdf')